In [5]:
import pandas as pd

The data was captured in the kaggle from: https://www.kaggle.com/datasets/athu1105/book-genre-prediction?resource=download

In [7]:
Data_frame = pd.read_csv('data.csv')
Data_frame.head()

,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...


# data have to be balanced

EDA and NLP Aug is for augmentation of imbalanced data

In [8]:
def finding_missing_data(data_frame, column):
    data_frame_null = data_frame[column].isnull()
    data_frame_nan = data_frame[column].isna()
    missing_null = data_frame_null.sum()
    missing_nan = data_frame_nan.sum()
    print('number of null values in %s is %i' % (column, missing_null))
    print('number of na values in %s is %i' % (column, missing_nan))


In [9]:
attrs = ['title', 'genre', 'summary']
for attr in attrs:
    finding_missing_data(Data_frame, attr)

number of null values in title is 0
number of na values in title is 0
number of null values in genre is 0
number of na values in genre is 0
number of null values in summary is 0
number of na values in summary is 0
